#Create tables and insert data
We'll begin by writing Python code in Google Colab that:

Sets up the database using sqlite3

Creates the necessary tables: LOCATION, DEPARTMENT, JOB, and EMPLOYEE

Inserts the data provided in the PDF

In [1]:
# Import necessary library
import sqlite3
import pandas as pd

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create LOCATION table
cursor.execute('''
CREATE TABLE LOCATION (
    Location_ID INTEGER PRIMARY KEY,
    City TEXT
);
''')

# Insert data into LOCATION table
cursor.executemany('''
INSERT INTO LOCATION (Location_ID, City)
VALUES (?, ?);
''', [
    (122, 'New York'),
    (123, 'Dallas'),
    (124, 'Chicago'),
    (167, 'Boston')
])

# Create DEPARTMENT table
cursor.execute('''
CREATE TABLE DEPARTMENT (
    Department_Id INTEGER PRIMARY KEY,
    Name TEXT,
    Location_Id INTEGER,
    FOREIGN KEY(Location_Id) REFERENCES LOCATION(Location_ID)
);
''')

# Insert data into DEPARTMENT table
cursor.executemany('''
INSERT INTO DEPARTMENT (Department_Id, Name, Location_Id)
VALUES (?, ?, ?);
''', [
    (10, 'Accounting', 122),
    (20, 'Sales', 124),
    (30, 'Research', 123),
    (40, 'Operations', 167)
])

# Create JOB table
cursor.execute('''
CREATE TABLE JOB (
    Job_ID INTEGER PRIMARY KEY,
    Designation TEXT
);
''')

# Insert data into JOB table
cursor.executemany('''
INSERT INTO JOB (Job_ID, Designation)
VALUES (?, ?);
''', [
    (667, 'Clerk'),
    (668, 'Staff'),
    (669, 'Analyst'),
    (670, 'Sales Person'),
    (671, 'Manager'),
    (672, 'President')
])

# Create EMPLOYEE table
cursor.execute('''
CREATE TABLE EMPLOYEE (
    Employee_Id INTEGER PRIMARY KEY,
    Last_Name TEXT,
    First_Name TEXT,
    Middle_Name TEXT,
    Job_Id INTEGER,
    Hire_Date TEXT,
    Salary INTEGER,
    Comm INTEGER,
    Department_Id INTEGER,
    FOREIGN KEY(Job_Id) REFERENCES JOB(Job_ID),
    FOREIGN KEY(Department_Id) REFERENCES DEPARTMENT(Department_Id)
);
''')

# Insert data into EMPLOYEE table
cursor.executemany('''
INSERT INTO EMPLOYEE (Employee_Id, Last_Name, First_Name, Middle_Name, Job_Id, Hire_Date, Salary, Comm, Department_Id)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
''', [
    (7369, 'Smith', 'John', 'Q', 667, '1984-12-17', 800, None, 20),
    (7499, 'Allen', 'Kevin', 'J', 670, '1985-02-20', 1600, 300, 30),
    (755, 'Doyle', 'Jean', 'K', 671, '1985-04-04', 2850, None, 30),
    (756, 'Dennis', 'Lynn', 'S', 671, '1985-05-15', 2750, None, 30),
    (757, 'Baker', 'Leslie', 'D', 671, '1985-06-10', 2200, None, 40),
    (7521, 'Wark', 'Cynthia', 'D', 670, '1985-02-22', 1250, 50, 30)
])


# Simple Queries
1. List all the employee details

In [2]:
# Query all data from EMPLOYEE table
df = pd.read_sql_query("SELECT * FROM EMPLOYEE;", conn)
df

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,755,Doyle,Jean,K,671,1985-04-04,2850,NaN,30
1,756,Dennis,Lynn,S,671,1985-05-15,2750,NaN,30
2,757,Baker,Leslie,D,671,1985-06-10,2200,NaN,40
3,7369,Smith,John,Q,667,1984-12-17,800,NaN,20
4,7499,Allen,Kevin,J,670,1985-02-20,1600,300.0,30
5,7521,Wark,Cynthia,D,670,1985-02-22,1250,50.0,30


In [3]:
# 2. List all the department details

df = pd.read_sql_query("SELECT * FROM DEPARTMENT;", conn)
df


,Department_Id,Name,Location_Id
0,10,Accounting,122
1,20,Sales,124
2,30,Research,123
3,40,Operations,167


In [4]:
# 3. List all job details

df = pd.read_sql_query("SELECT * FROM JOB;", conn)
df

,Job_ID,Designation
0,667,Clerk
1,668,Staff
2,669,Analyst
3,670,Sales Person
4,671,Manager
5,672,President


In [6]:
# 4. List all the locations
df = pd.read_sql_query("SELECT * FROM LOCATION;", conn)
df

,Location_ID,City
0,122,New York
1,123,Dallas
2,124,Chicago
3,167,Boston


In [7]:
# 5. List out the First Name, Last Name, Salary, Commission for all Employees
df = pd.read_sql_query("""
SELECT First_Name, Last_Name, Salary, Comm#
FROM EMPLOYEE;
""", conn)
df


,First_Name,Last_Name,Salary,Comm
0,Jean,Doyle,2850,NaN
1,Lynn,Dennis,2750,NaN
2,Leslie,Baker,2200,NaN
3,John,Smith,800,NaN
4,Kevin,Allen,1600,300.0
5,Cynthia,Wark,1250,50.0


In [8]:
#6. List out the Employee ID, Last Name, Department ID with alias

df = pd.read_sql_query("""
SELECT
    Employee_Id AS "ID of the Employee",
    Last_Name AS "Name of the Employee",
    Department_Id AS "Dep_id"
FROM EMPLOYEE;
""", conn)
df


,ID of the Employee,Name of the Employee,Dep_id
0,755,Doyle,30
1,756,Dennis,30
2,757,Baker,40
3,7369,Smith,20
4,7499,Allen,30
5,7521,Wark,30


In [9]:
# 7. List out the annual salary of the employees with their names only
# (Annual salary = Salary * 12)

df = pd.read_sql_query("""
SELECT
    First_Name || ' ' || Last_Name AS "Employee Name",
    Salary * 12 AS "Annual Salary"
FROM EMPLOYEE;
""", conn)
df


,Employee Name,Annual Salary
0,Jean Doyle,34200
1,Lynn Dennis,33000
2,Leslie Baker,26400
3,John Smith,9600
4,Kevin Allen,19200
5,Cynthia Wark,15000


# WHERE Clause Queries

In [10]:
# 1. List the details about "Smith"

# Get all employee details where the last name is 'Smith'
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Last_Name = 'Smith';
""", conn)
df


# This query filters the employee table to return only the rows where the last name is exactly "Smith".

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,7369,Smith,John,Q,667,1984-12-17,800,None,20


In [12]:
# 2. List out the employees who are working in department 20

# Get employees who belong to Department_Id = 20
df1 = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Department_Id = 20;
""", conn)
df1


,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,7369,Smith,John,Q,667,1984-12-17,800,None,20


In [13]:
# 3. List out the employees who are earning salary between 2000 and 3000

# Get employees with salary in the range 2000 to 3000 (inclusive)
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Salary BETWEEN 2000 AND 3000;
""", conn)
df

# BETWEEN is inclusive and filters for salaries ≥ 2000 and ≤ 3000.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,755,Doyle,Jean,K,671,1985-04-04,2850,None,30
1,756,Dennis,Lynn,S,671,1985-05-15,2750,None,30
2,757,Baker,Leslie,D,671,1985-06-10,2200,None,40


In [14]:
# 4. List out the employees who are working in department 10 or 20

# Get employees from department 10 or 20
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Department_Id IN (10, 20);
""", conn)
df

# IN is used for checking multiple possible values — it's more readable than using multiple OR

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,7369,Smith,John,Q,667,1984-12-17,800,None,20


In [15]:
# 5. Find out the employees who are not working in department 10 or 30

# Get employees excluding those in department 10 and 30
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Department_Id NOT IN (10, 30);
""", conn)
df

# Explanation: NOT IN filters out rows with the specified values.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,757,Baker,Leslie,D,671,1985-06-10,2200,None,40
1,7369,Smith,John,Q,667,1984-12-17,800,None,20


In [16]:
# 6. List out the employees whose name starts with 'L'

# Use LIKE with 'L%' to match names starting with L
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE First_Name LIKE 'L%';
""", conn)
df

# Explanation: LIKE 'L%' matches names starting with L (% means any characters can follow).

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,756,Dennis,Lynn,S,671,1985-05-15,2750,None,30
1,757,Baker,Leslie,D,671,1985-06-10,2200,None,40


In [17]:
# 7. List out the employees whose name starts with 'L' and ends with 'E'

# First name should start with L and end with E
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE First_Name LIKE 'L%E';
""", conn)
df

# Explanation: LIKE 'L%E' matches names that start with L and end with E (with anything in between).

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,757,Baker,Leslie,D,671,1985-06-10,2200,None,40


In [18]:
# 8. List out the employees whose name length is 4 and start with 'J'

# Filter First_Name of length 4 and starting with J
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE LENGTH(First_Name) = 4 AND First_Name LIKE 'J%';
""", conn)
df

#  Explanation: LENGTH() returns number of characters in a string. Combined with LIKE 'J%',
#  it filters names starting with J and exactly 4 characters long.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,755,Doyle,Jean,K,671,1985-04-04,2850,None,30
1,7369,Smith,John,Q,667,1984-12-17,800,None,20


In [19]:
# 9. List out the employees who are working in department 30 and draw the salaries more than 2500

# Use AND to apply both conditions
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Department_Id = 30 AND Salary > 2500;
""", conn)
df

# Explanation: This filters rows that meet both criteria using the AND operator.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,755,Doyle,Jean,K,671,1985-04-04,2850,None,30
1,756,Dennis,Lynn,S,671,1985-05-15,2750,None,30


In [20]:
# 10. List out the employees who are not receiving commission

# Check for NULL in Comm (no commission)
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Comm IS NULL;
""", conn)
df

# Explanation: IS NULL is used to filter rows where the commission column has no value (i.e., not getting commission).

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,755,Doyle,Jean,K,671,1985-04-04,2850,None,30
1,756,Dennis,Lynn,S,671,1985-05-15,2750,None,30
2,757,Baker,Leslie,D,671,1985-06-10,2200,None,40
3,7369,Smith,John,Q,667,1984-12-17,800,None,20


# ORDER BY Clause Queries

These queries help us sort the results based on one or more columns.



In [21]:
# 1. List out the Employee ID and Last Name in ascending order based on the Employee ID

# Select specific columns and sort by Employee_Id in ascending order (default)
df = pd.read_sql_query("""
SELECT Employee_Id, Last_Name
FROM EMPLOYEE
ORDER BY Employee_Id;
""", conn)
df

# Explanation: ORDER BY Employee_Id sorts results in ascending order (ASC is default).

,Employee_Id,Last_Name
0,755,Doyle
1,756,Dennis
2,757,Baker
3,7369,Smith
4,7499,Allen
5,7521,Wark


In [22]:
# 2. List out the Employee ID and Name in descending order based on salary

# Concatenate first and last names and sort by Salary in descending order
df = pd.read_sql_query("""
SELECT Employee_Id, First_Name || ' ' || Last_Name AS Name, Salary
FROM EMPLOYEE
ORDER BY Salary DESC;
""", conn)
df

# Explanation: DESC means descending order. We use || to combine first and last names into one column called Name.

,Employee_Id,Name,Salary
0,755,Jean Doyle,2850
1,756,Lynn Dennis,2750
2,757,Leslie Baker,2200
3,7499,Kevin Allen,1600
4,7521,Cynthia Wark,1250
5,7369,John Smith,800


In [23]:
# 3. List out the employee details according to their Last Name in ascending order

# Sort all employee records by Last_Name A → Z
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
ORDER BY Last_Name;
""", conn)
df

# Explanation: Alphabetical ordering is done here by the last name.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,7499,Allen,Kevin,J,670,1985-02-20,1600,300.0,30
1,757,Baker,Leslie,D,671,1985-06-10,2200,NaN,40
2,756,Dennis,Lynn,S,671,1985-05-15,2750,NaN,30
3,755,Doyle,Jean,K,671,1985-04-04,2850,NaN,30
4,7369,Smith,John,Q,667,1984-12-17,800,NaN,20
5,7521,Wark,Cynthia,D,670,1985-02-22,1250,50.0,30


In [24]:
# 4. List out the employee details according to their Last Name in ascending order and then Department ID in descending order

# First sort by Last_Name (ASC), then by Department_Id (DESC)
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
ORDER BY Last_Name ASC, Department_Id DESC;
""", conn)
df
#  Explanation: You can sort on multiple columns.
#  This sorts first alphabetically by last name and, for duplicates, by department ID in reverse order.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,7499,Allen,Kevin,J,670,1985-02-20,1600,300.0,30
1,757,Baker,Leslie,D,671,1985-06-10,2200,NaN,40
2,756,Dennis,Lynn,S,671,1985-05-15,2750,NaN,30
3,755,Doyle,Jean,K,671,1985-04-04,2850,NaN,30
4,7369,Smith,John,Q,667,1984-12-17,800,NaN,20
5,7521,Wark,Cynthia,D,670,1985-02-22,1250,50.0,30


#  GROUP BY and HAVING Clause Queries

These are used for aggregating data (like counting, summing, or averaging) based on groups (e.g., departments or job roles).

In [25]:
# 1. List out the department-wise maximum salary, minimum salary, and average salary of the employees

# Aggregate salary stats grouped by department
df = pd.read_sql_query("""
SELECT
    Department_Id,
    MAX(Salary) AS Max_Salary,
    MIN(Salary) AS Min_Salary,
    AVG(Salary) AS Avg_Salary
FROM EMPLOYEE
GROUP BY Department_Id;
""", conn)
df

# Explanation: GROUP BY clusters employees by department, then aggregates their salary using MAX, MIN, and AVG.

,Department_Id,Max_Salary,Min_Salary,Avg_Salary
0,20,800,800,800.0
1,30,2850,1250,2112.5
2,40,2200,2200,2200.0


In [26]:
# 2. List out the job-wise maximum salary, minimum salary, and average salary of the employees

# Same logic as before, but grouped by Job_Id
df = pd.read_sql_query("""
SELECT
    Job_Id,
    MAX(Salary) AS Max_Salary,
    MIN(Salary) AS Min_Salary,
    AVG(Salary) AS Avg_Salary
FROM EMPLOYEE
GROUP BY Job_Id;
""", conn)
df

# Explanation: We group employees by Job_Id and calculate salary statistics for each role.

,Job_Id,Max_Salary,Min_Salary,Avg_Salary
0,667,800,800,800.0
1,670,1600,1250,1425.0
2,671,2850,2200,2600.0


In [27]:
# 3. List out the number of employees who joined each month in ascending order

# Extract month and count how many joined in each month
df = pd.read_sql_query("""
SELECT
    strftime('%m', Hire_Date) AS Month,
    COUNT(*) AS Employee_Count
FROM EMPLOYEE
GROUP BY Month
ORDER BY Month;
""", conn)
df

# Explanation: strftime('%m', Hire_Date) extracts the month from the date. We then count employees by month.

,Month,Employee_Count
0,02,2
1,04,1
2,05,1
3,06,1
4,12,1


In [28]:
# 4. List out the number of employees for each month and year in ascending order based on the year and month

# Extract year and month, and group accordingly
df = pd.read_sql_query("""
SELECT
    strftime('%Y', Hire_Date) AS Year,
    strftime('%m', Hire_Date) AS Month,
    COUNT(*) AS Employee_Count
FROM EMPLOYEE
GROUP BY Year, Month
ORDER BY Year, Month;
""", conn)
df

# Explanation: We extract both year and month from the hire date and count employees accordingly.

,Year,Month,Employee_Count
0,1984,12,1
1,1985,02,2
2,1985,04,1
3,1985,05,1
4,1985,06,1


In [29]:
# 5. List out the Department ID having at least four employees

# Group by department and filter with HAVING
df = pd.read_sql_query("""
SELECT
    Department_Id,
    COUNT(*) AS Employee_Count
FROM EMPLOYEE
GROUP BY Department_Id
HAVING COUNT(*) >= 4;
""", conn)
df

# Explanation: HAVING is used after grouping to filter aggregated results (unlike WHERE which works on individual rows).

,Department_Id,Employee_Count
0,30,4


In [30]:
# 6. How many employees joined in February month

# Use WHERE to filter month = '02'
df = pd.read_sql_query("""
SELECT COUNT(*) AS Employees_In_February
FROM EMPLOYEE
WHERE strftime('%m', Hire_Date) = '02';
""", conn)
df

# Explanation: This filters only February joinings using strftime().



,Employees_In_February
0,2


In [31]:
# 7. How many employees joined in May or June month

# Filter using IN clause
df = pd.read_sql_query("""
SELECT COUNT(*) AS Employees_In_May_Or_June
FROM EMPLOYEE
WHERE strftime('%m', Hire_Date) IN ('05', '06');
""", conn)
df

# Explanation: IN ('05', '06') checks for both May and June.

,Employees_In_May_Or_June
0,2


In [32]:
# 8. How many employees joined in 1985?

# Filter only year = '1985'
df = pd.read_sql_query("""
SELECT COUNT(*) AS Employees_1985
FROM EMPLOYEE
WHERE strftime('%Y', Hire_Date) = '1985';
""", conn)
df

#  Explanation: Year is extracted and filtered just like month.

,Employees_1985
0,5


In [33]:
# 9. How many employees joined each month in 1985?

# Group only 1985 joinings by month
df = pd.read_sql_query("""
SELECT
    strftime('%m', Hire_Date) AS Month,
    COUNT(*) AS Employee_Count
FROM EMPLOYEE
WHERE strftime('%Y', Hire_Date) = '1985'
GROUP BY Month
ORDER BY Month;
""", conn)
df

# Explanation: We apply WHERE to get only 1985 data, then group it by month.

,Month,Employee_Count
0,02,2
1,04,1
2,05,1
3,06,1


In [34]:
# 10. How many employees were joined in April 1985?

# Filter for specific month and year
df = pd.read_sql_query("""
SELECT COUNT(*) AS April_1985_Joinings
FROM EMPLOYEE
WHERE strftime('%Y-%m', Hire_Date) = '1985-04';
""", conn)
df

# Explanation: We match year and month together using strftime('%Y-%m').

,April_1985_Joinings
0,1


In [35]:
# 11. Which is the Department ID having ≥ 3 employees joining in April 1985?

# Combine filtering, grouping, and HAVING
df = pd.read_sql_query("""
SELECT Department_Id, COUNT(*) AS Emp_Count
FROM EMPLOYEE
WHERE strftime('%Y-%m', Hire_Date) = '1985-04'
GROUP BY Department_Id
HAVING COUNT(*) >= 3;
""", conn)
df

# Explanation: We narrow down to April 1985, then count employees per department and use HAVING to filter those with 3 or more joinings.

,Department_Id,Emp_Count


# JOINs Queries

 where we combine information from multiple tables to gain richer insights

In [36]:
# 1. List out employees with their department names

# Join EMPLOYEE with DEPARTMENT using Department_Id
df = pd.read_sql_query("""
SELECT
    E.Employee_Id,
    E.First_Name || ' ' || E.Last_Name AS Employee_Name,
    D.Name AS Department_Name
FROM EMPLOYEE E
JOIN DEPARTMENT D ON E.Department_Id = D.Department_Id;
""", conn)
df

# Explanation: We join EMPLOYEE and DEPARTMENT on the common key Department_Id to get the department name for each employee.

,Employee_Id,Employee_Name,Department_Name
0,755,Jean Doyle,Research
1,756,Lynn Dennis,Research
2,757,Leslie Baker,Operations
3,7369,John Smith,Sales
4,7499,Kevin Allen,Research
5,7521,Cynthia Wark,Research


In [37]:
# 2. Display employees with their designations

# Join EMPLOYEE with JOB table to get the job title (designation)
df = pd.read_sql_query("""
SELECT
    E.Employee_Id,
    E.First_Name || ' ' || E.Last_Name AS Employee_Name,
    J.Designation
FROM EMPLOYEE E
JOIN JOB J ON E.Job_Id = J.Job_ID;
""", conn)
df

# Explanation: This joins EMPLOYEE with JOB to show each employee's role.

,Employee_Id,Employee_Name,Designation
0,755,Jean Doyle,Manager
1,756,Lynn Dennis,Manager
2,757,Leslie Baker,Manager
3,7369,John Smith,Clerk
4,7499,Kevin Allen,Sales Person
5,7521,Cynthia Wark,Sales Person


In [38]:
# 3. Display the employees with their department names and city

# Join EMPLOYEE → DEPARTMENT → LOCATION to get City and Department
df = pd.read_sql_query("""
SELECT
    E.Employee_Id,
    E.First_Name || ' ' || E.Last_Name AS Employee_Name,
    D.Name AS Department_Name,
    L.City
FROM EMPLOYEE E
JOIN DEPARTMENT D ON E.Department_Id = D.Department_Id
JOIN LOCATION L ON D.Location_Id = L.Location_ID;
""", conn)
df

# Explanation: This uses chained JOINs across three tables to get city and department info for each employee.

,Employee_Id,Employee_Name,Department_Name,City
0,755,Jean Doyle,Research,Dallas
1,756,Lynn Dennis,Research,Dallas
2,757,Leslie Baker,Operations,Boston
3,7369,John Smith,Sales,Chicago
4,7499,Kevin Allen,Research,Dallas
5,7521,Cynthia Wark,Research,Dallas


In [39]:
# 4. How many employees are working in different departments? Display with department names

# Group by department and count number of employees
df = pd.read_sql_query("""
SELECT
    D.Name AS Department_Name,
    COUNT(E.Employee_Id) AS Employee_Count
FROM EMPLOYEE E
JOIN DEPARTMENT D ON E.Department_Id = D.Department_Id
GROUP BY D.Name;
""", conn)
df

# Explanation: We group by department name and count how many employees are in each.

,Department_Name,Employee_Count
0,Operations,1
1,Research,4
2,Sales,1


In [40]:
# 5. How many employees are working in the Sales department?

# Filter after joining on department name = 'Sales'
df = pd.read_sql_query("""
SELECT COUNT(*) AS Sales_Employee_Count
FROM EMPLOYEE E
JOIN DEPARTMENT D ON E.Department_Id = D.Department_Id
WHERE D.Name = 'Sales';
""", conn)
df

# Explanation: We join EMPLOYEE and DEPARTMENT and filter only the Sales department.

,Sales_Employee_Count
0,1


In [41]:
# 6. Which is the department having ≥ 3 employees? Display the department names in ascending order

# Group and filter departments with 3 or more employees
df = pd.read_sql_query("""
SELECT
    D.Name AS Department_Name,
    COUNT(E.Employee_Id) AS Employee_Count
FROM EMPLOYEE E
JOIN DEPARTMENT D ON E.Department_Id = D.Department_Id
GROUP BY D.Name
HAVING COUNT(E.Employee_Id) >= 3
ORDER BY D.Name;
""", conn)
df

# Explanation: After grouping, we filter using HAVING and sort the department names alphabetically.

,Department_Name,Employee_Count
0,Research,4


In [42]:
# 7. How many employees are working in 'Dallas'?

# Join across LOCATION and count employees in Dallas
df = pd.read_sql_query("""
SELECT COUNT(*) AS Employees_In_Dallas
FROM EMPLOYEE E
JOIN DEPARTMENT D ON E.Department_Id = D.Department_Id
JOIN LOCATION L ON D.Location_Id = L.Location_ID
WHERE L.City = 'Dallas';
""", conn)
df
#  Explanation: We join all three tables and filter only for city = Dallas.

,Employees_In_Dallas
0,4


In [43]:
# 8. Display all employees in Sales or Operation departments

# Filter for specific departments using IN
df = pd.read_sql_query("""
SELECT
    E.Employee_Id,
    E.First_Name || ' ' || E.Last_Name AS Employee_Name,
    D.Name AS Department_Name
FROM EMPLOYEE E
JOIN DEPARTMENT D ON E.Department_Id = D.Department_Id
WHERE D.Name IN ('Sales', 'Operations');
""", conn)
df

# Explanation: We fetch employees whose department name is either 'Sales' or 'Operations'.



,Employee_Id,Employee_Name,Department_Name
0,757,Leslie Baker,Operations
1,7369,John Smith,Sales


# Conditional Statement Queries

Here, we use SQL's CASE statement to create custom logic — like salary grading based on salary ranges.

In [44]:
# 1. Display the employee details with salary grades. Use conditional statement to create a grade column

# Add a custom 'Grade' column based on salary ranges using CASE
df = pd.read_sql_query("""
SELECT
    Employee_Id,
    First_Name || ' ' || Last_Name AS Employee_Name,
    Salary,
    CASE
        WHEN Salary >= 3000 THEN 'A'
        WHEN Salary >= 2000 THEN 'B'
        WHEN Salary >= 1000 THEN 'C'
        ELSE 'D'
    END AS Grade
FROM EMPLOYEE;
""", conn)
df

# Explanation: CASE lets us create a new column called Grade based on salary brackets.

,Employee_Id,Employee_Name,Salary,Grade
0,755,Jean Doyle,2850,B
1,756,Lynn Dennis,2750,B
2,757,Leslie Baker,2200,B
3,7369,John Smith,800,D
4,7499,Kevin Allen,1600,C
5,7521,Cynthia Wark,1250,C


In [45]:
# 2. List out the number of employees grade-wise. Use conditional statement to create a grade column

# Group by the derived grade using CASE and count employees
df = pd.read_sql_query("""
SELECT
    CASE
        WHEN Salary >= 3000 THEN 'A'
        WHEN Salary >= 2000 THEN 'B'
        WHEN Salary >= 1000 THEN 'C'
        ELSE 'D'
    END AS Grade,
    COUNT(*) AS Employee_Count
FROM EMPLOYEE
GROUP BY Grade
ORDER BY Grade;
""", conn)
df
# Explanation: This query groups employees by the grade categories and counts them.

,Grade,Employee_Count
0,B,3
1,C,2
2,D,1


In [46]:
# 3. Display the employee salary grades and the number of employees between 2000 to 5000 range of salary

# First filter employees with salary in the 2000–5000 range, then group by grade
df = pd.read_sql_query("""
SELECT
    CASE
        WHEN Salary >= 4000 THEN 'A'
        WHEN Salary >= 3000 THEN 'B'
        WHEN Salary >= 2000 THEN 'C'
    END AS Grade,
    COUNT(*) AS Employee_Count
FROM EMPLOYEE
WHERE Salary BETWEEN 2000 AND 5000
GROUP BY Grade
ORDER BY Grade;
""", conn)
df
# Explanation: We first apply a WHERE filter for salaries between 2000–5000 and then assign and group by grade.

,Grade,Employee_Count
0,C,3


# Subqueries

Subqueries are queries inside another query, often used to fetch intermediate results that are used in the main query.

In [47]:
# 1. Display the employees list who got the maximum salary

# Subquery gets max salary; outer query finds employee(s) with that salary
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Salary = (
    SELECT MAX(Salary) FROM EMPLOYEE
);
""", conn)
df
# Explanation: The inner query finds the max salary. The outer query fetches employees matching that salary.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,755,Doyle,Jean,K,671,1985-04-04,2850,None,30


In [48]:
# 2. Display the employees who are working in the sales department

# Subquery finds Department_Id for 'Sales'
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Department_Id = (
    SELECT Department_Id FROM DEPARTMENT WHERE Name = 'Sales'
);
""", conn)
df
# Explanation: We use a subquery to get the department ID for 'Sales', then fetch employees from that department.



,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,7369,Smith,John,Q,667,1984-12-17,800,None,20


In [49]:
# 3. Display the employees who are working as 'Clerk'

# Subquery gets Job_ID for 'Clerk'
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Job_Id = (
    SELECT Job_ID FROM JOB WHERE Designation = 'Clerk'
);
""", conn)
df
# Explanation: We find the job ID of 'Clerk' and use that in the main query.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,7369,Smith,John,Q,667,1984-12-17,800,None,20


In [50]:
# 4. Display the list of employees who are living in 'Boston'

# Join EMPLOYEE → DEPARTMENT → LOCATION via subquery
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Department_Id IN (
    SELECT Department_Id
    FROM DEPARTMENT
    WHERE Location_Id = (
        SELECT Location_ID FROM LOCATION WHERE City = 'Boston'
    )
);
""", conn)
df
# Explanation: This nested subquery setup traces employees who belong to departments located in Boston.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,757,Baker,Leslie,D,671,1985-06-10,2200,None,40


In [51]:
# 5. Find out the number of employees working in the sales department

df = pd.read_sql_query("""
SELECT COUNT(*) AS Sales_Employee_Count
FROM EMPLOYEE
WHERE Department_Id = (
    SELECT Department_Id FROM DEPARTMENT WHERE Name = 'Sales'
);
""", conn)
df
# Explanation: We use a subquery to find Sales' department ID and count employees in that department.



,Sales_Employee_Count
0,1


In [52]:
# 6. Update the salaries of employees who are working as clerks on the basis of 10%

# Increase salary by 10% for clerks (read + update example)
cursor.execute("""
UPDATE EMPLOYEE
SET Salary = Salary * 1.10
WHERE Job_Id = (
    SELECT Job_ID FROM JOB WHERE Designation = 'Clerk'
);
""")
conn.commit()

# Check updated values
df = pd.read_sql_query("""
SELECT * FROM EMPLOYEE WHERE Job_Id = (
    SELECT Job_ID FROM JOB WHERE Designation = 'Clerk'
);
""", conn)
df
#  Explanation: We update salaries by multiplying them by 1.10 for all employees with the job 'Clerk'.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,7369,Smith,John,Q,667,1984-12-17,880.0,None,20


In [53]:
# 7. Display the second highest salary drawing employee details

df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Salary = (
    SELECT MAX(Salary)
    FROM EMPLOYEE
    WHERE Salary < (
        SELECT MAX(Salary) FROM EMPLOYEE
    )
);
""", conn)
df
# Explanation: We find the second highest salary by excluding the max salary and then taking the next max.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,756,Dennis,Lynn,S,671,1985-05-15,2750,None,30


In [54]:
# 8. List out the employees who earn more than every employee in department 30

df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE
WHERE Salary > (
    SELECT MAX(Salary)
    FROM EMPLOYEE
    WHERE Department_Id = 30
);
""", conn)
df
# Explanation: This finds employees whose salary is greater than the highest salary in department 30.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id


In [55]:
# 9. Find out which department has no employees

df = pd.read_sql_query("""
SELECT *
FROM DEPARTMENT
WHERE Department_Id NOT IN (
    SELECT DISTINCT Department_Id FROM EMPLOYEE
);
""", conn)
df
# Explanation: We look for department IDs that are not assigned to any employee.

,Department_Id,Name,Location_Id
0,10,Accounting,122


In [56]:
# 10. Find out the employees who earn greater than the average salary for their department

# Correlated subquery: inner query depends on outer query’s Department_Id
df = pd.read_sql_query("""
SELECT *
FROM EMPLOYEE E1
WHERE Salary > (
    SELECT AVG(Salary)
    FROM EMPLOYEE E2
    WHERE E1.Department_Id = E2.Department_Id
);
""", conn)
df
# Explanation: For each employee, we compare their salary to the average salary of their own department.

,Employee_Id,Last_Name,First_Name,Middle_Name,Job_Id,Hire_Date,Salary,Comm,Department_Id
0,755,Doyle,Jean,K,671,1985-04-04,2850,None,30
1,756,Dennis,Lynn,S,671,1985-05-15,2750,None,30
